In [ ]:
import cv2
import numpy as np

# Load pre-trained YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize the webcam (0 refers to the default camera)
cap = cv2.VideoCapture(0)

# Create a small window with a specified name
cv2.namedWindow("Live Detection", cv2.WINDOW_NORMAL)

while True:
    # Capture each frame from the camera
    ret, frame = cap.read()

    # If frame is not successfully captured, break out of the loop
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to grayscale for face and pedestrian detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process YOLO detections
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression (NMS) to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)  # Adjust NMS threshold as needed

    # Capture the detected faces and objects (No rectangles drawn here)
    detected_faces = []
    detected_objects = []

    # Collect detected faces
    for (x, y, w, h) in faces:
        detected_faces.append((x, y, w, h))  # Store coordinates of detected faces

    # Collect detected objects
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence_text = f'{round(confidences[i] * 100, 2)}%'  # Display confidence percentage
            detected_objects.append((label, confidence_text, (x, y, w, h)))

    # Prepare the box for displaying detected objects information
    output_text = ""
    for obj in detected_objects:
        output_text += f"{obj[0]}: {obj[1]}\n"

    # Draw the small box on the right side of the screen
    if output_text:
        height, width, _ = frame.shape
        # Create a small box area on the right side of the screen
        box_width = 250  # Adjust width for longer text
        box_height = 200  # Adjust height for text
        box_x = width - box_width - 10  # Position box at the right side
        box_y = 20  # Distance from the top

        # Draw a rectangle to create the box
        cv2.rectangle(frame, (box_x, box_y), (box_x + box_width, box_y + box_height), (0, 0, 0), -1)

        # Place the output text inside the small box
        cv2.putText(frame, output_text, (box_x + 10, box_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Optionally, you can process the detected_faces and detected_objects as needed
    for face in detected_faces:
        print(f"Detected face: {face}")
    for obj in detected_objects:
        print(f"Detected object: {obj[0]} with confidence: {obj[1]} at position: {obj[2]}")

    # Display the frame without resizing
    cv2.imshow("Live Detection", frame)

    # Move the window to the right side of the screen
    cv2.moveWindow("Live Detection", 1000, 100)  # Adjust the window position as needed

    # Exit the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Detected face: (316, 277, 117, 117)
Detected object: person with confidence: 99.44% at position: (217, 218, 373, 261)
Detected object: person with confidence: 99.74% at position: (222, 215, 366, 262)
Detected object: person with confidence: 99.8% at position: (219, 215, 368, 261)
Detected object: person with confidence: 99.86% at position: (222, 218, 366, 260)
Detected object: person with confidence: 99.71% at position: (220, 217, 368, 262)
Detected object: person with confidence: 99.66% at position: (215, 213, 376, 266)
Detected object: person with confidence: 99.67% at position: (211, 212, 381, 266)
Detected object: person with confidence: 99.83% at position: (212, 212, 382, 265)
Detected object: person with confidence: 99.31% at position: (208, 211, 389, 268)
Detected object: person with confidence: 99.82% at position: (214, 212, 380, 265)
Detected object: person with confidence: 99.8% at position: (214, 213, 382, 266)
Detected object: person with confidence: 99.81% at position: (21